# Assignment 2 - Geopandas

Create a map that visualizes the preferences of cities/municipalities between Jollibee and Mcdonalds in 'Metropolitan Manila'. Your map should distinctly show:
1. The locations of the Jollibee and Mcdonalds Establishments
1. The "preference" between the two
1. The bottom two cities in terms of number or restaurants/population

In [1]:
import warnings
warnings.filterwarnings("ignore")

import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransform

from svgpathtools import svg2paths
from svgpath2mpl import parse_path


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-ktz_7b2l because the default path (/home/rbanquerigo/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
conn = psycopg2.connect(dbname="postgis", 
                 user="gsa2022", 
                 password="g5!V%T1Vmd", 
                 host="192.168.212.99", 
                 port=32771)

## Data Preparation

Get the shape of Metro Manila

In [3]:
mm_shp = gpd.read_postgis(""" 
SELECT * FROM gadm.ph
WHERE name_1 = 'Metropolitan Manila'
""", con=conn, geom_col='geom')
mm_shp.head(5)

gid gid_0       name_0     gid_1               name_1 nl_name_1  \
0  973   PHL  Philippines  PHL.47_1  Metropolitan Manila      None   
1  974   PHL  Philippines  PHL.47_1  Metropolitan Manila      None   
2  965   PHL  Philippines  PHL.47_1  Metropolitan Manila      None   
3  966   PHL  Philippines  PHL.47_1  Metropolitan Manila      None   
4  967   PHL  Philippines  PHL.47_1  Metropolitan Manila      None   

         gid_2       name_2                       varname_2 nl_name_2  \
0  PHL.47.13_1      Pateros                            None      None   
1  PHL.47.14_1  Quezon City                            None      None   
2   PHL.47.5_1  Mandaluyong            Mandaluyong, City of      None   
3   PHL.47.6_1       Manila  City of Manila|Manila, City of      None   
4   PHL.47.7_1     Marikina               Marikina, City of      None   

            type_2 engtype_2    cc_2    hasc_2  \
0  Lungsod|Siyudad      City  137606  PH.MM.PT   
1  Lungsod|Siyudad      City  137404  PH.MM.QE   
2  Lungsod|Siyudad      City  137401  PH.MM.MD   
3  Lungsod|Siyudad      City  133900  PH.MM.MN   
4  Lungsod|Siyudad      City  137402  PH.MM.MR   

                                                geom  
0  MULTIPOLYGON (((121.06864 14.53992, 121.06883 ...  
1  MULTIPOLYGON (((121.01576 14.60610, 121.01511 ...  
2  MULTIPOLYGON (((121.05405 14.57747, 121.05408 ...  
3  MULTIPOLYGON (((120.99142 14.56218, 120.98849 ...  
4  MULTIPOLYGON (((121.07346 14.61962, 121.07432 ...

Get all points where Jollibee store is tagged in Metro Manila using name, addr:housename and brand in ph_point as our columns.

In [4]:
joli_pts = gpd.read_postgis(""" 
SELECT p.*, g.* FROM ph_point as p
INNER JOIN gadm.ph as g
ON ST_WITHIN(p.way, g.geom)
WHERE 
(lower(p.name)  like '%jollibee%'
OR
lower(p."addr:housename")  like '%jollibee%'
OR
lower(p.brand) like '%jollibee%')
AND
g.name_1 = 'Metropolitan Manila'
""", con=conn, geom_col='way')
display(joli_pts.head(5))

osm_id access addr:housename addr:housenumber addr:interpolation  \
0  3721647225   None           None             None               None   
1  2197156471   None           None             None               None   
2  4187178106   None           None             None               None   
3   821585644   None          Shell             None               None   
4   255053274   None           None             None               None   

  admin_level aerialway aeroway    amenity  area  ... nl_name_1       gid_2  \
0        None      None    None  fast_food  None  ...      None  PHL.47.8_1   
1        None      None    None  fast_food  None  ...      None  PHL.47.8_1   
2        None      None    None  fast_food  None  ...      None  PHL.47.8_1   
3        None      None    None  fast_food  None  ...      None  PHL.47.8_1   
4        None      None    None  fast_food  None  ...      None  PHL.47.2_1   

       name_2            varname_2 nl_name_2           type_2 engtype_2  \
0  Muntinlupa  Muntinlupa, City of      None  Lungsod|Siyudad      City   
1  Muntinlupa  Muntinlupa, City of      None  Lungsod|Siyudad      City   
2  Muntinlupa  Muntinlupa, City of      None  Lungsod|Siyudad      City   
3  Muntinlupa  Muntinlupa, City of      None  Lungsod|Siyudad      City   
4   Las Piñas   Las Piñas, City of      None  Lungsod|Siyudad      City   

     cc_2    hasc_2                                               geom  
0  137603  PH.MM.MU  0106000020E61000000100000001030000000100000014...  
1  137603  PH.MM.MU  0106000020E61000000100000001030000000100000014...  
2  137603  PH.MM.MU  0106000020E61000000100000001030000000100000014...  
3  137603  PH.MM.MU  0106000020E61000000100000001030000000100000014...  
4  137601  PH.MM.LP  0106000020E6100000020000000103000000010000004A...  

[5 rows x 84 columns]

Get all points where McDonald's store is tagged in Metro Manila using name, addr:housename and brand in ph_point as our columns. I only used the first 4 letters assuming these are all McDonalds restaurant.

In [5]:

mc_pts = gpd.read_postgis(""" 
SELECT p.*,g.* FROM ph_point as p
INNER JOIN gadm.ph as g
ON ST_WITHIN(p.way, g.geom)
WHERE 
(lower(substr(p.name, 1, 4)) like '%mcdo%'
OR
lower(substr(p."addr:housename", 1, 4)) like '%mcdo%'
OR
lower(substr(p.brand, 1, 4)) like '%mcdo%')
AND
g.name_1 = 'Metropolitan Manila'
""", con=conn, geom_col='way')
display(mc_pts.head(5))

osm_id access addr:housename addr:housenumber addr:interpolation  \
0   929096991   None           None             None               None   
1  2705236200   None           None             None               None   
2  4532131491   None           None             None               None   
3   255067248   None           None             None               None   
4  1203902763   None           None             None               None   

  admin_level aerialway aeroway    amenity  area  ... nl_name_1        gid_2  \
0        None      None    None  fast_food  None  ...      None  PHL.47.13_1   
1        None      None    None  fast_food  None  ...      None  PHL.47.14_1   
2        None      None    None  fast_food  None  ...      None  PHL.47.14_1   
3        None      None    None  fast_food  None  ...      None  PHL.47.14_1   
4        None      None    None  fast_food  None  ...      None  PHL.47.14_1   

        name_2 varname_2 nl_name_2           type_2 engtype_2    cc_2  \
0      Pateros      None      None  Lungsod|Siyudad      City  137606   
1  Quezon City      None      None  Lungsod|Siyudad      City  137404   
2  Quezon City      None      None  Lungsod|Siyudad      City  137404   
3  Quezon City      None      None  Lungsod|Siyudad      City  137404   
4  Quezon City      None      None  Lungsod|Siyudad      City  137404   

     hasc_2                                               geom  
0  PH.MM.PT  0106000020E610000001000000010300000001000000A3...  
1  PH.MM.QE  0106000020E61000000100000001030000000100000084...  
2  PH.MM.QE  0106000020E61000000100000001030000000100000084...  
3  PH.MM.QE  0106000020E61000000100000001030000000100000084...  
4  PH.MM.QE  0106000020E61000000100000001030000000100000084...  

[5 rows x 84 columns]

Get the count of fast_food and restaurants amenities per city within Metro Manila and sort it by increasing order.

In [6]:
sql = (""" 
SELECT g.name_2, count(*) FROM ph_point as p
INNER JOIN gadm.ph as g
ON ST_WITHIN(p.way, g.geom)
WHERE 
(amenity LIKE '%fast_food%' OR
amenity LIKE '%restaurant%')
AND
name_1 = 'Metropolitan Manila'
GROUP BY 1
ORDER BY 2
""")
df_res = pd.read_sql_query(sql, conn)

display(df_res.head(5))

name_2  count
0        Navotas      3
1        Pateros     20
2        Malabon     32
3         Taguig     45
4  Kalookan City     55

Load our financial_pop.csv file from our previous class to get the density per city against each population and sort it in increasing value. In this case we can see Navotas and Kalookan City has the bottom 2 lowest density.

In [7]:
df_pop = pd.read_csv('financial_pop.csv')
df_res = df_res.merge(df_pop,
                      left_on='name_2',
                      right_on='shp_municipality',
                      how='left')

df_res = df_res[['name_2', 'count', 'pop']]
df_res['restaurant_density'] = df_res['count'] / df_res['pop']
df_res = df_res.sort_values(by='restaurant_density', ascending=True)
display(df_res.head(5))

name_2  count        pop  restaurant_density
0        Navotas      3   249463.0            0.000012
4  Kalookan City     55  1583978.0            0.000035
3         Taguig     45   804915.0            0.000056
2        Malabon     32   365525.0            0.000088
5     Valenzuela     70   620422.0            0.000113

Prepare markers for our points, Jollibee and McDo will have its own markers.

In [8]:
j_path, j_att = svg2paths('jabi.svg')
mc_path, mc_att = svg2paths('mcdo.svg')

j_mark = parse_path(j_att[0]['d'])
mc_mark = parse_path(mc_att[0]['d'])

j_mark.vertices -= j_mark.vertices.mean(axis=0)
mc_mark.vertices -= mc_mark.vertices.mean(axis=0)

mc_mark = mc_mark.transformed(mtransform.Affine2D().rotate_deg(180))
mc_mark = mc_mark.transformed(mtransform.Affine2D().scale(-1,1))

j_mark = j_mark.transformed(mtransform.Affine2D().rotate_deg(180))
j_mark = j_mark.transformed(mtransform.Affine2D().scale(-1,1))

## Final Plot

- Plotting of the map of Metro Manila 
- Plotting of the points for Jollibee and McDo 
- Color coding of the preferences as described in the legend
- Using hatch to mark the bottom two cities

Conclusion: There's a lot of data missing for the stores. Since there are McDo and Jollibee stores in Navotas. For the density of population I don't have an estimate but I don't think Caloocan should be the 2nd with lowest density I think pateros may come in within top 2 or some other city like Valenzuela or Malabon given how developed Caloocan is.

In [9]:
ax = mm_shp.plot(figsize=(30, 30), edgecolor='black')

for i in mm_shp.name_2.unique():
    m_len = len(mc_pts[mc_pts.name_2==i])
    j_len = len(joli_pts[joli_pts.name_2==i])
    hatch = ''
    if m_len == 0 and j_len == 0:
        c_col = 'lightgray'
    elif m_len > j_len:
        c_col = 'lightgoldenrodyellow'
    elif m_len < j_len:
        c_col = 'mistyrose'
    else:
        c_col = 'white'
    
    if i in df_res.name_2[0:2].tolist(): hatch = '||'
    mm_shp[mm_shp.name_2==i].plot(ax=ax, color=c_col, edgecolor='black', hatch=hatch)
    
joli_pts.plot(ax=ax, color='red', markersize=300, alpha=1, marker=j_mark)
mc_pts.plot(ax=ax, color='gold', markersize=300, alpha=1, marker=mc_mark)   

mm_shp['coords'] = mm_shp['geom'].apply(lambda x: x.representative_point().coords[:])
mm_shp['coords'] = [coords[0] for coords in mm_shp['coords']]
for idx, row in mm_shp.iterrows():
                 plt.annotate(row['name_2'],
                 xy=row['coords'],
                 horizontalalignment='center',
                 fontsize=10)
lst_patch = []
lst_patch.append(mpatches.Patch(color='mistyrose', label='Jollibee Preference'))
lst_patch.append(mpatches.Patch(color='lightgoldenrodyellow', label='McDo Preference'))
lst_patch.append(mpatches.Patch(color='white', label='No Preference'))
lst_patch.append(mpatches.Patch(color='lightgray', label='No Branches'))
lst_patch.append(mpatches.Patch(facecolor='#DCDCDC', hatch='||', label='Bottom 2 Cities # of Restaurant/Fast Food'))
ax.legend(handles=lst_patch, loc='upper left');
